In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7v09xi72
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7v09xi72
  Resolved https://github.com/huggingface/transformers to commit 897a826d830e8b1e03eb482b165b5d88a7a08d5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 34.5 MB

In [ ]:
url = "https://www.alfabank.by/credits/cards/100-day/#kak_poluchit"

In [ ]:
questions = ["Как получить карту альфа-банка \"100 дней\"?", "Какие документы необходимы для получения карты альфа-банка \"100 дней\"?", "Какие требования к получателю карты альфа-банка \"100 дней\"?"]

In [ ]:
answers = []

In [ ]:

import requests
from lxml import etree

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"
}

# Получаем HTML-страницу по URL с использованием заголовков
response = requests.get(url, headers=headers)
html = response.content

# Проверяем статус ответа
if response.status_code == 200:
    # Создаем объект ElementTree для парсинга HTML
    tree = etree.HTML(html)

    # Используем XPath для извлечения текста элементов
    answers.append(tree.xpath('//*[@id="kak_poluchit"]/section/div[1]/div[2]/p/text()')[0])
    answers.append(tree.xpath('//*[@id="kak_poluchit"]/section/div[2]/div[2]/ul/li/text()')[0])
    answers.append('\n'.join(tree.xpath('//*[@id="kak_poluchit"]/section/div[3]/div[2]')[0].itertext()))
else:
    print("Ошибка при получении страницы. Код статуса:", response.status_code)

In [ ]:
import pandas as pd
pd_dataset = pd.DataFrame({"questions":questions, "answers":answers})

In [ ]:
pd_dataset

,questions,answers
0,"Как получить карту альфа-банка ""100 дней""?","Вы можете оформить карту ""100 дней"" по телефон..."
1,Какие документы необходимы для получения карты...,"Паспорт гражданина РБ, вид на жительство в РБ ..."
2,Какие требования к получателю карты альфа-банк...,Резидент Республики Беларусь\n.\nВозраст:\n от...


In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                              "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                        )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
prompt = questions[0]

prompt_template=f'''[INST]
    {prompt}[/INST]
    '''
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.1, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512, early_stopping=True)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:400: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> [INST]
    Как получить карту альфа-банка "100 дней"?[/INST]
    1. Введите в поисковую строку "100 дней карта альфа-банка".
2. Нажмите на первый результат, который соответствует карте "100 дней" альфа-банка.
3. Нажмите на кнопку "Получить".
4. Введите ваш номер телефона и ваш адрес электронной почты.
5. Нажмите на кнопку "Получить".
6. Ваш код почты и смс-код будут отправлены вам на указанные адреса.
7. Введите коды в поле, которое появилось на экране.
8. Нажмите на кнопку "Получить".
9. Ваша карта "100 дней" альфа-банка будет отправлена вам в течение 2-3 дней.</s>


In [ ]:
pd_dataset["text"] = pd_dataset[["questions", "answers"]].apply(lambda x: "<s> [INST]" + x["questions"] + "[/INST] " +x["answers"]+" </s>", axis=1)
print(pd_dataset.iloc[0])
data = Dataset.from_pandas(pd_dataset)


questions           Как получить карту альфа-банка "100 дней"?
answers      Вы можете оформить карту "100 дней" по телефон...
text         <s> [INST]Как получить карту альфа-банка "100 ...
Name: 0, dtype: object


In [ ]:
    model.config.use_cache=False
    model.config.pretraining_tp=1
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    peft_config = LoraConfig(
        r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
    )
    model = get_peft_model(model, peft_config)
    training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-alfa",
        per_device_train_batch_size=64,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=20,
        num_train_epochs=1,
        max_steps=300,
        fp16=True
    )
    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )
    trainer.train()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): QuantLinear(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (quant_linear_module): Q

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
20,1.317500
40,0.233200
60,0.135600
80,0.129300
100,0.121200
120,0.119700
140,0.034200
160,0.023900
180,0.021500
200,0.021200


TrainOutput(global_step=300, training_loss=0.15062875886758167, metrics={'train_runtime': 1203.0258, 'train_samples_per_second': 15.96, 'train_steps_per_second': 0.249, 'total_flos': 167111059046400.0, 'train_loss': 0.15062875886758167, 'epoch': 300.0})